In a different notebook, kaiserOptimizationWithoutCcd, ascvd, non cv death, and dementia intercepts were optimized to 
represent the average increased risk of the Kaiser population without taking into account WMH-specific information (eg hazard ratios
for WMH severity, SBI etc). The goal of this notebook is to test whether the optimization was done correctly. The outcome incidence rates we obtain here are very close to the reference rates (note: the reference rates were obtained from a different simulation/notebook).

In [1]:
import numpy as np
import pandas as pd
import os
import importlib.util
from scipy import optimize
import matplotlib.pyplot as plt

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)
dataDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/NOTEBOOKS/DATA"

pd.set_option('future.no_silent_downcasting', True) 

from microsim.outcome import OutcomeType
from microsim.population_factory import PopulationFactory
from microsim.dementia_model_repository import DementiaModelRepository
from microsim.dementia_model import DementiaModel
from microsim.cv_model import CVModelMale, CVModelFemale
from microsim.non_cv_death_model import NonCVDeathModel

In [2]:
class DementiaModelRepositoryKaiser(DementiaModelRepository):
    def __init__(self, wmhSpecific=False):
        super().__init__(wmhSpecific=False)
        
module = importlib.import_module("microsim.outcome_model_repository")
setattr(module, "DementiaModelRepository", DementiaModelRepositoryKaiser)

In [3]:
class NonCVDeathModelKaiser(NonCVDeathModel):
    def __init__(self, wmhSpecific):
            super().__init__(wmhSpecific=False)

module = importlib.import_module("microsim.non_cv_model_repository")
setattr(module, "NonCVDeathModel", NonCVDeathModelKaiser)

In [4]:
class CVModelMaleKaiser(CVModelMale):
    def __init__(self, wmhSpecific):
            super().__init__(wmhSpecific=False)

class CVModelFemaleKaiser(CVModelFemale):
    def __init__(self, wmhSpecific):
            super().__init__(wmhSpecific=False)

module = importlib.import_module("microsim.cv_model_repository")
setattr(module, "CVModelMale", CVModelMaleKaiser)
setattr(module, "CVModelFemale", CVModelFemaleKaiser)

In [5]:
popSize = 300000
duration=4
pop = PopulationFactory.get_kaiser_population(n=popSize)
pop.advance(duration, nWorkers=1)

In [6]:
strokeRates = pop.get_outcome_incidence_rates_at_end_of_wave(outcomesTypeList=[OutcomeType.STROKE], 
                                                                                     wave=duration-1)
dementiaRates = pop.get_outcome_incidence_rates_at_end_of_wave(outcomesTypeList=[OutcomeType.DEMENTIA], 
                                                                                       wave=duration-1)
deathRates = pop.get_outcome_incidence_rates_at_end_of_wave(outcomesTypeList=[OutcomeType.DEATH], 
                                                                                    wave=duration-1)
miRates = pop.get_outcome_incidence_rates_at_end_of_wave(outcomesTypeList=[OutcomeType.MI], 
                                                                                     wave=duration-1)
rates = {"stroke": strokeRates, "dementia": dementiaRates, "death": deathRates, "mi": miRates}

In [7]:
strokeRatesRef = 12 #microsim simulation

deathRatesRef = 27 #microsim simulation

dementiaRatesRef = 11 #microsim simulation

ratesRef = {"stroke": strokeRatesRef, "death": deathRatesRef, "dementia": dementiaRatesRef, "mi": 12}

In [8]:
print("     Group     Reference     Simulation")
for outcome in rates.keys():
    print(f"{outcome:>10}" +
        f"{ratesRef[outcome]:>14.1f} " + 
          f"{rates[outcome]:>14.1f}")

     Group     Reference     Simulation
    stroke          12.0           11.7
  dementia          11.0           10.9
     death          27.0           26.8
        mi          12.0           11.5
